# Merging talks to their speakers

There are a few phases to this stage the of the data cleaning. 
* First we split the speakers of the talks into their own columns. Along the way, there are a few steps that are completed manually
* Second, we merge the meta data of the talks with the descriptions of the speakers 

### Step 0 - Importing packages

In [1]:
# Set of imports
import pandas as pd
import csv
import string
import numpy as np

In [4]:
# Import talk file 
talks = pd.read_csv('TEDonly.csv', encoding='utf-8')
speakers = pd.read_csv('speakers.csv',encoding = 'utf-8')

In [5]:
talks.head()

,Unnamed: 0,rowID,Talk_ID,public_url,speaker_name,headline,description,event,duration,published,tags,views,text
0,0,0,1,https://www.ted.com/talks/al_gore_on_averting_...,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",3266733,"Thank you so much, Chris. And it's truly a g..."
1,1,1,7,https://www.ted.com/talks/david_pogue_says_sim...,David Pogue,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",1702201,"(Music: ""The Sound of Silence,"" Simon & Garf..."
2,2,2,53,https://www.ted.com/talks/majora_carter_s_tale...,Majora Carter,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",2000421,If you're here today — and I'm very happy th...
3,3,3,66,https://www.ted.com/talks/ken_robinson_says_sc...,Ken Robinson,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",51614087,Good morning. How are you? (Laughter) ...
4,4,4,92,https://www.ted.com/talks/hans_rosling_shows_t...,Hans Rosling,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",12662135,"About 10 years ago, I took on the task to te..."


### Step 1 - Begin splitting and cleaning of the talks file

In [6]:
#Drop weird unnamed column - 
talks.drop(talks.columns[0:2], axis=1, inplace=True)
talks.head()

,Talk_ID,public_url,speaker_name,headline,description,event,duration,published,tags,views,text
0,1,https://www.ted.com/talks/al_gore_on_averting_...,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",3266733,"Thank you so much, Chris. And it's truly a g..."
1,7,https://www.ted.com/talks/david_pogue_says_sim...,David Pogue,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",1702201,"(Music: ""The Sound of Silence,"" Simon & Garf..."
2,53,https://www.ted.com/talks/majora_carter_s_tale...,Majora Carter,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",2000421,If you're here today — and I'm very happy th...
3,66,https://www.ted.com/talks/ken_robinson_says_sc...,Ken Robinson,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",51614087,Good morning. How are you? (Laughter) ...
4,92,https://www.ted.com/talks/hans_rosling_shows_t...,Hans Rosling,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",12662135,"About 10 years ago, I took on the task to te..."


In [7]:
# Break speakers into two columns
splitList = r' \+ | , | and '
#https://stackoverflow.com/questions/37543724/python-regex-for-finding-all-words-in-a-string

splitSpeakers = talks['speaker_name'].str.split(splitList, expand=True).rename(columns=lambda x: f"speaker_{x+1}")

# Join the speakers to the talks dataframe and drop the original speakers:
splitTalks = talks.join(splitSpeakers)



In [8]:
# Fix incorrect splitting of this talk's speakers
splitTalks.at[289,"speaker_1"] = splitTalks.at[289,"speaker_name"]
splitTalks.at[289,"speaker_2"] = None
splitTalks.iloc[289]

Talk_ID                                                       466
public_url      https://www.ted.com/talks/astonishing_performa...
speaker_name    Gustavo Dudamel and the Teresa Carreño Youth O...
headline                         El Sistema's top youth orchestra
description     The Teresa Carreño Youth Orchestra contains th...
event                                                     TED2009
duration                                                  0:17:06
published                                                 2/18/09
tags            conducting,TED Prize,entertainment,children,li...
views                                                     2165588
text              Chris Anderson: And now we go live to Caraca...
speaker_1       Gustavo Dudamel and the Teresa Carreño Youth O...
speaker_2                                                    None
Name: 289, dtype: object

In [9]:
splitTalks.drop(["speaker_name"], axis=1, inplace=True)

#### Check for non-unicode characters in both talks and the speakers

In [91]:
test_inds = splitTalks["speaker_1"].apply(lambda x: len([True for i in str(x) if (ord(i) < 32 or ord(i) > 122)]) > 0)
#https://stackoverflow.com/questions/36340627/removing-non-ascii-characters-and-
#                                   replacing-with-spaces-from-pandas-data-frame

#https://blog.teamtreehouse.com/python-single-line-loops

In [92]:
sinds = speakers["name"].apply(lambda x: len([True for i in str(x) if (ord(i) < 32 or ord(i) > 122)]) > 0)

In [93]:
# Check for matches across the rows with special characters in talks and 
# the rows with special characters in speakers

st = splitTalks[test_inds][["speaker_1"]]
speaks = speakers[sinds][["name"]]

name_test = pd.merge(st, speaks, how = "outer", left_on = "speaker_1", right_on = "name", indicator = True)
# https://stackoverflow.com/questions/20375561/joining-pandas-dataframes-by-column-names

In [94]:
# This is the number of special character rows that are properly merged:
len(name_test[name_test["_merge"] == "both"])

7

In [95]:
# Save the ones that we need to manually edit - 
name_test.to_csv('./speakers_work/manual_name_edits_old.csv', sep = ',')

In [96]:
# Save the talks file where it is. 
splitTalks.to_csv('./speakers_work/TEDonly_splitSpeakers.csv',sep = ',')

At this point, we do a manual edit, fixing the names that were parsed strangely due to the html interpretor. The file `manual_name_edits.csv` shows which names have accents and/or special characters and whether they are matched correctly between the talks and speakers files. During this cleaning, we renamed this file `TEDonly_splitSpeakers_clean.csv`.

In [99]:
# reload the splitTalks:
clean_talks = pd.read_csv('./speakers_work/TEDonly_splitSpeakers_clean.csv',encoding = 'utf-8')
clean_talks.drop(clean_talks.columns[0], axis=1, inplace=True)

In [100]:
# Merge the cleaned_talks with the speakers file. Then clean up the column names. 
result1 = pd.merge(clean_talks, speakers, 
                   how = 'left', left_on = 'speaker_1', right_on = 'name')
result1.drop(['name'], axis=1, inplace=True)
result1.rename(columns={'occupation':'speaker1_occupation', 
                        'introduction':'speaker1_introduction', 
                        'profile':'speaker1_profile'}, inplace=True)

# https://stackoverflow.com/questions/35321812/move-column-in-pandas-dataframe/35321983
# pop off the speaker_2 column and put at the end of the dataframe
cols = list(result1.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('speaker_2'))
result1 = result1[cols+['speaker_2']]
result1.head()

,Talk_ID,public_url,headline,description,event,duration,published,tags,views,text,speaker_1,speaker1_occupation,speaker1_introduction,speaker1_profile,speaker_2
0,1,https://www.ted.com/talks/al_gore_on_averting_...,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",3266733,"Thank you so much, Chris. And it's truly a g...",Al Gore,Climate advocate,Nobel Laureate Al Gore focused the world’s att...,Why you should listen\nFormer Vice President A...,NaN
1,7,https://www.ted.com/talks/david_pogue_says_sim...,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",1702201,"(Music: ""The Sound of Silence,"" Simon & Garf...",David Pogue,Technology columnist,David Pogue is the personal technology columni...,Why you should listen\nWhich cell phone to cho...,NaN
2,53,https://www.ted.com/talks/majora_carter_s_tale...,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",2000421,If you're here today — and I'm very happy th...,Majora Carter,Activist for environmental justice,Majora Carter redefined the field of environme...,Why you should listen\nMajora Carter is a visi...,NaN
3,66,https://www.ted.com/talks/ken_robinson_says_sc...,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",51614087,Good morning. How are you? (Laughter) ...,Ken Robinson,Author/educator,Creativity expert Sir Ken Robinson challenges ...,Why you should listen\nWhy don't we get the be...,NaN
4,92,https://www.ted.com/talks/hans_rosling_shows_t...,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",12662135,"About 10 years ago, I took on the task to te...",Hans Rosling,Global health expert; data visionary,"In Hans Rosling’s hands, data sings. Global tr...",Why you should listen\nEven the most worldly a...,NaN


In [113]:
result2 = pd.merge(result1, speakers, 
                   how = 'left', left_on = 'speaker_2', right_on = 'name')
result2.rename(columns={'occupation':'speaker2_occupation', 
                        'introduction':'speaker2_introduction', 
                        'profile':'speaker2_profile'}, inplace=True)
result2.drop(['name'], axis=1, inplace=True)
result2

,Talk_ID,public_url,headline,description,event,duration,published,tags,views,text,speaker_1,speaker1_occupation,speaker1_introduction,speaker1_profile,speaker_2,speaker2_occupation,speaker2_introduction,speaker2_profile
0,1,https://www.ted.com/talks/al_gore_on_averting_...,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",3266733,"Thank you so much, Chris. And it's truly a g...",Al Gore,Climate advocate,Nobel Laureate Al Gore focused the world’s att...,Why you should listen\nFormer Vice President A...,NaN,NaN,NaN,NaN
1,7,https://www.ted.com/talks/david_pogue_says_sim...,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",1702201,"(Music: ""The Sound of Silence,"" Simon & Garf...",David Pogue,Technology columnist,David Pogue is the personal technology columni...,Why you should listen\nWhich cell phone to cho...,NaN,NaN,NaN,NaN
2,53,https://www.ted.com/talks/majora_carter_s_tale...,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",2000421,If you're here today — and I'm very happy th...,Majora Carter,Activist for environmental justice,Majora Carter redefined the field of environme...,Why you should listen\nMajora Carter is a visi...,NaN,NaN,NaN,NaN
3,66,https://www.ted.com/talks/ken_robinson_says_sc...,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",51614087,Good morning. How are you? (Laughter) ...,Ken Robinson,Author/educator,Creativity expert Sir Ken Robinson challenges ...,Why you should listen\nWhy don't we get the be...,NaN,NaN,NaN,NaN
4,92,https://www.ted.com/talks/hans_rosling_shows_t...,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",12662135,"About 10 years ago, I took on the task to te...",Hans Rosling,Global health expert; data visionary,"In Hans Rosling’s hands, data sings. Global tr...",Why you should listen\nEven the most worldly a...,NaN,NaN,NaN,NaN
5,96,https://www.ted.com/talks/tony_robbins_asks_wh...,Why we do what we do,"Tony Robbins discusses the ""invisible forces"" ...",TED2006,0:21:45,6/27/06,"entertainment,goal-setting,potential,psycholog...",22368699,Thank you. I have to tell you I'm both chall...,Tony Robbins,Life coach; expert in leadership psychology,Tony Robbins makes it his business to know why...,Why you should listen\nTony Robbins might have...,NaN,NaN,NaN,NaN
6,49,https://www.ted.com/talks/joshua_prince_ramus_...,Behind the design of Seattle's library,Architect Joshua Prince-Ramus takes the audien...,TED2006,0:19:58,7/10/06,"library,architecture,design,culture,collaboration",1042335,I'm going to present three projects in rapid...,Joshua Prince-Ramus,Architect,Joshua Prince-Ramus is best known as architect...,Why you should listen\nWith one of the decade'...,NaN,NaN,NaN,NaN
7,86,https://www.ted.com/talks/julia_sweeney_on_let...,Letting go of God,When two young Mormon missionaries knock on Ju...,TED2006,0:16:32,7/10/06,"atheism,Christianity,religion,God,comedy,humor...",3903747,"On September 10, the morning of my seventh b...",Julia Sweeney,"Actor, comedian, playwright",Julia Sweeney creates comedic works that tackl...,Why you should listen\nJulia Sweeney is a writ...,NaN,NaN,NaN,NaN
8,71,https://www.ted.com/talks/rick_warren_on_a_lif...,A life of purpose,"Pastor Rick Warren, author of ""The Purpose-Dri...",TED2006,0:21:02,7/18/06,"Christianity,philanthropy,religion,God,happine...",3361934,"I'm often asked, ""What surprised you about t...",Rick Warren,"Pastor, author",Pastor Rick Warren is the author of The Purpos...,Why you should listen\nPastor Rick Warren is o...,NaN,NaN,NaN,NaN
9,94,https://www.

In [117]:
result2.to_csv('./speakers_work/TEDonly_speakers_doubles.csv', sep = ',')

In [114]:
print(clean_talks.shape)
print(result2.shape)

(992, 12)
(997, 18)


In [115]:
dup_first = result2["headline"].duplicated(keep='first')
dup_second= result2["headline"].duplicated(keep='last')
double_talks = pd.concat([result2[dup_first], result2[dup_second]], axis = 0)

In [116]:
double_talks.to_csv('./speakers_work/doubles.csv', sep = ',')

Again we need to do a manual step here. We have two authors named Chris Anderson and two named Michael Green. Python cannot tell which one is the correct speaker to attach to the talk, so it duplicates the row associated to the talks given by people with these names. We call this new file `TEDonly_speakers.csv` and this file is the near final data file for the talks at the main TED event (i.e. those called TED YYYY). 

## Check the file that all speakers and meta speaker information is in place. 

In [5]:
ts_final = pd.read_csv('./speakers_work/TEDonly_speakers.csv')
ts_final.drop(ts_final.columns[0], axis=1, inplace=True)

In [17]:
s1cut = ts_final[ts_final['speaker1_occupation'].isnull()]

s2inds = ts_final[~ts_final['speaker_2'].isnull()].index
#ts_final.iloc[s2inds]
s2cut = ts_final.iloc[s2inds].loc[ts_final.iloc[s2inds]['speaker2_occupation'].isnull()]
#s2inds

In [20]:
s_missing = pd.concat([s1cut,s2cut], axis = 0)
s_missing = s_missing.drop_duplicates(keep = "first")
s_missing.to_csv('./speakers_work/missing_meta.csv', sep = ',')

The file `missing_meta.csv` contains the rows of our dataset that are missing the speaker meta information. In this last step we will do one final manual set of additions to fill these missing cells in. This final file will be called `TEDonly_speakers_final.csv`.

To ensure that all of the cleaning steps are carried backwards to the early files, we will make `TEDonly_final.csv` that includes everything in `TEDonly_speakers_final.csv` save for the meta-information about the speakers. 

In [10]:
ts = pd.read_csv('./speakers_work/TEDonly_speakers_final.csv',sep = ',')

In [11]:
# remove speaker meta information except the name
cols = list(ts.columns.values)
s1 = cols.index('speaker_1')
s2 = cols.index('speaker_2')
s3 = cols.index('speaker_3')
s4 = cols.index('speaker_4')
new_cols = cols[0:s1+1]+['speaker_2']+['speaker_3']+['speaker_4']

In [12]:
just_talks = ts[new_cols]
just_talks.to_csv('./speakers_work/TEDonly_final.csv', sep = ',')

*Note*: The above code was adjusted after the data cleaning was complete due to reorganizing of the directories within this github.